# Это неудачный опыт фич-инжеренинга на геоданных. если выцепите от сюда что-то полезное - кайф

In [ ]:
!pip install h3

In [2]:
!pip install osmnx

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 33.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: shapely
    Found existing installation: Shapely 1.8.5.post1
    Uninstalling Shapely-1.8.5.post1:
      Successfully uninstalled Shapely-1.8.5.post1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.


In [3]:
import osmnx as ox
import pandas as pd
import numpy as np
import folium
import h3
from geopy.distance import geodesic
import geopandas as gpd
from shapely.geometry import Point
from geopandas.tools import sjoin

In [4]:
# Определим функцию для выполнения запроса
def osm_query(tag, city):
    # Получаем объекты с нужным тегом в заданном городе
    gdf = ox.geometries_from_place(city, tag).reset_index()
    gdf['city'] = np.full(len(gdf), city.split(',')[0])
    gdf['object'] = np.full(len(gdf), list(tag.keys())[0])
    gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])
    gdf = gdf[['city', 'object', 'type', 'geometry']]
    print(f"Found {gdf.shape[0]} results for {tag} in {city}")
    return gdf

# Тег для поиска всех кафе
tag = {'amenity': 'cafe'}

city = 'Москва, Россия'

# Выполним запрос и получим все кофейни в указанном городе
cafe_gdf = osm_query(tag, city)

#cafes_gdf = cafes_gdf.to_crs("EPSG:3857")
cafe_gdf['lat'] = cafe_gdf['geometry'].centroid.y
cafe_gdf['lon'] = cafe_gdf['geometry'].centroid.x

cafe_gdf.head()

/tmp/ipykernel_30/1920592931.py:4: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index()


Found 4856 results for {'amenity': 'cafe'} in Москва, Россия


/tmp/ipykernel_30/1920592931.py:21: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cafe_gdf['lat'] = cafe_gdf['geometry'].centroid.y
/tmp/ipykernel_30/1920592931.py:22: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  cafe_gdf['lon'] = cafe_gdf['geometry'].centroid.x


,city,object,type,geometry,lat,lon
0,Москва,amenity,cafe,POINT (36.86793 55.43079),55.430795,36.867931
1,Москва,amenity,cafe,POINT (37.35671 55.57183),55.571830,37.356709
2,Москва,amenity,cafe,POINT (37.20662 55.37346),55.373459,37.206619
3,Москва,amenity,cafe,POINT (37.26576 55.41391),55.413913,37.265759
4,Москва,amenity,cafe,POINT (37.33219 55.50160),55.501602,37.332194


In [5]:
cafe_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4856 entries, 0 to 4855
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   city      4856 non-null   object  
 1   object    4856 non-null   object  
 2   type      4856 non-null   object  
 3   geometry  4856 non-null   geometry
 4   lat       4856 non-null   float64 
 5   lon       4856 non-null   float64 
dtypes: float64(2), geometry(1), object(3)
memory usage: 227.8+ KB


In [6]:
import osmnx as ox
import geopandas as gpd
import numpy as np

# Определим функцию для выполнения запроса OSM по тегам
def osm_query(tags, city):
    # Пустой датафрейм для хранения результатов
    gdf_combined = gpd.GeoDataFrame()
    
    # Цикл по всем тегам из списка
    for tag in tags:
        try:
            # Получаем объекты с нужным тегом в заданном городе
            gdf = ox.geometries_from_place(city, tag).reset_index()
            gdf['city'] = np.full(len(gdf), city.split(',')[0])
            gdf['object'] = np.full(len(gdf), list(tag.keys())[0])
            gdf['type'] = np.full(len(gdf), tag[list(tag.keys())[0]])
            gdf = gdf[['city', 'object', 'type', 'geometry']]
            
            # Выводим количество найденных результатов
            print(f"Found {gdf.shape[0]} results for {tag} in {city}")
            
            # Объединяем результаты с общим датафреймом
            gdf_combined = pd.concat([gdf_combined, gdf], ignore_index=True)
            
        except Exception as e:
            print(f"Error fetching data for tag {tag}: {e}")
    
    return gdf_combined

# Список тегов для поиска зданий и других объектов
tags = [
    {'building': 'apartments'}, {'building': 'detached'}, 
    {'building': 'dormitory'}, {'building': 'hotel'}, 
    {'building': 'house'}, {'building': 'semidetached_house'}, 
    {'building': 'terrace'}, {'building': 'commercial'},
    {'building': 'office'}, {'building': 'retail'}, 
    {'building': 'train_station'}
]

# Указываем город
city = 'Москва, Россия'

# Выполняем запросы по всем тегам и объединяем результаты
buildings_gdf = osm_query(tags, city)

# Преобразуем геометрию в координаты широты и долготы
buildings_gdf['lat'] = buildings_gdf['geometry'].centroid.y
buildings_gdf['lon'] = buildings_gdf['geometry'].centroid.x

# Просмотр первых строк результата
buildings_gdf.head()

/tmp/ipykernel_30/166561062.py:14: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index()


Found 35908 results for {'building': 'apartments'} in Москва, Россия


/tmp/ipykernel_30/166561062.py:14: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index()


Found 1810 results for {'building': 'detached'} in Москва, Россия


/tmp/ipykernel_30/166561062.py:14: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index()


...

Found 4800 results for {'building': 'retail'} in Москва, Россия


/tmp/ipykernel_30/166561062.py:14: FutureWarning: The `geometries` module and `geometries_from_X` functions have been renamed the `features` module and `features_from_X` functions. Use these instead. The `geometries` module and function names are deprecated and will be removed in the v2.0.0 release. See the OSMnx v2 migration guide: https://github.com/gboeing/osmnx/issues/1123
  gdf = ox.geometries_from_place(city, tag).reset_index()


Found 281 results for {'building': 'train_station'} in Москва, Россия


/tmp/ipykernel_30/166561062.py:48: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_gdf['lat'] = buildings_gdf['geometry'].centroid.y
/tmp/ipykernel_30/166561062.py:49: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  buildings_gdf['lon'] = buildings_gdf['geometry'].centroid.x


,city,object,type,geometry,lat,lon
0,Москва,building,apartments,"POLYGON ((37.22605 55.51590, 37.22610 55.51601...",55.515882,37.226577
1,Москва,building,apartments,"POLYGON ((37.22435 55.51618, 37.22440 55.51629...",55.516155,37.224920
2,Москва,building,apartments,"POLYGON ((37.22453 55.51664, 37.22459 55.51675...",55.516603,37.225122
3,Москва,building,apartments,"POLYGON ((37.22477 55.51708, 37.22482 55.51719...",55.517047,37.225350
4,Москва,building,apartments,"POLYGON ((37.29724 55.48611, 37.29828 55.48612...",55.486054,37.297759


In [7]:
buildings_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 85822 entries, 0 to 85821
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   city      85822 non-null  object  
 1   object    85822 non-null  object  
 2   type      85822 non-null  object  
 3   geometry  85822 non-null  geometry
 4   lat       85822 non-null  float64 
 5   lon       85822 non-null  float64 
dtypes: float64(2), geometry(1), object(3)
memory usage: 3.9+ MB


In [8]:
cafe_gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 4856 entries, 0 to 4855
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   city      4856 non-null   object  
 1   object    4856 non-null   object  
 2   type      4856 non-null   object  
 3   geometry  4856 non-null   geometry
 4   lat       4856 non-null   float64 
 5   lon       4856 non-null   float64 
dtypes: float64(2), geometry(1), object(3)
memory usage: 227.8+ KB


In [9]:
import geopandas as gpd

# Константа для радиуса Земли в метрах
EARTH_RADIUS = 6371000  

# Функция для расчета расстояния по формуле Хаверсина
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    return 2 * EARTH_RADIUS * np.arcsin(np.sqrt(a))

# Функция для поиска ближайшего кафе в радиусе 1 км
def find_nearest_cafe(building_row, cafes_df, max_distance=25, search_radius=1000):
    # Отбираем кафе, которые находятся в радиусе search_radius от здания
    nearby_cafes = cafes_df[
        haversine(building_row['lat'], building_row['lon'], cafes_df['lat'], cafes_df['lon']) <= search_radius
    ]
    
    nearest_cafe = None
    min_distance = float('inf')

    for _, cafe_row in nearby_cafes.iterrows():
        distance = haversine(building_row['lat'], building_row['lon'],
                             cafe_row['lat'], cafe_row['lon'])

        if distance < min_distance:
            min_distance = distance
            nearest_cafe = cafe_row

    is_in_sc = min_distance <= max_distance

    return pd.Series({
        'dist_ncafe': min_distance,
        'lat_ncafe': nearest_cafe['lat'] if nearest_cafe is not None else None,
        'lon_ncafe': nearest_cafe['lon'] if nearest_cafe is not None else None,
        'is_in_sc': is_in_sc
    })

# Применяем функцию для каждого здания
buildings_gdf[['dist_ncafe', 'lat_ncafe', 'lon_ncafe', 'is_in_sc']] = buildings_gdf.apply(
    lambda row: find_nearest_cafe(row, cafe_gdf), axis=1
)

KeyboardInterrupt: 

In [19]:
# Размер гексагона
size = 10

# Функция для получения h3 индекса
def take_h3_indx(lat, lon, size):
    return h3.latlng_to_cell(lat, lon, size)

# Применяем функцию для каждого здания
buildings_gdf['region_id'] = buildings_gdf.apply(lambda row: take_h3_indx(row['lat'], row['lon'], size), axis=1)

In [38]:
buildings_gdf

,city,object,type,geometry,lat,lon,region_id
0,Москва,building,apartments,POINT (37.57809 55.65122),55.651221,37.578088,8a11aa79405ffff
1,Москва,building,apartments,"POLYGON ((37.42680 55.84512, 37.42677 55.84523...",55.845690,37.426733,8a11aa722ccffff
2,Москва,building,apartments,"POLYGON ((37.42645 55.84645, 37.42641 55.84656...",55.847007,37.426367,8a11aa7221b7fff
3,Москва,building,apartments,"POLYGON ((37.42409 55.84758, 37.42427 55.84758...",55.847457,37.424205,8a11aa722c5ffff
4,Москва,building,apartments,"POLYGON ((37.42426 55.84787, 37.42443 55.84787...",55.847745,37.424370,8a11aa722c4ffff
...,...,...,...,...,...,...,...
51156,Москва,building,train_station,"POLYGON ((37.66834 55.83939, 37.66836 55.83941...",55.839422,37.668676,8a11aa62291ffff
51157,Москва,building,train_station,"POLYGON ((37.62422 55.76140, 37.62421 55.76140...",55.761528,37.624431,8a11aa7ab89ffff
51158,Москва,building,train_station,"POLYGON ((37.74700 55.80325, 37.74721 55.80325...",55.802939,37.747254,8a11aa615a2ffff
51159,Москва,building,train_station,"POLYGON ((37.62925 55.74255, 37.62934 55.74254...",55.742393,37.629258,8a11aa78c9a7fff
